<a href="https://colab.research.google.com/github/palepss/etl-process/blob/main/etl_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Processo de ETL: extração, limpeza e carregamento de dados.**

Objetivo geral: melhorar a qualidade do dataset, bem como, criar novas colunas de informações, para auxiliar no processo de tomada de decisão para evitar acidentes aéreos.

**Etapa 1: Extração**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

# dataset retirado no site https://dados.gov.br/dataset/ocorrencias-aeronauticas-da-aviacao-civil-brasileira/resource/9d8a7e09-5f75-47b3-891b-b5f1bab26d59
# Reading dataset as dataframe
df = pd.read_csv("/content/drive/My Drive/data_mining/ocorrencia - ocorrencia.csv", parse_dates=['ocorrencia_dia'], dayfirst=True) # You can also use "utf-8"
pd.set_option('display.max_colwidth', None) # Setting this so we can see the full content of cells
pd.set_option('display.max_columns', None) # to make sure we can see all the columns in output window
df.head(10)

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
5,39807,INCIDENTE,SALVADOR,BA,****,2010-01-06,17:53:00,0
6,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,INCIDENTE GRAVE,CANUTAMA,AM,****,2010-01-09,12:30:00,3
8,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,****,2010-01-10,20:00:00,0


In [3]:
df.dtypes

codigo_ocorrencia                    int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

In [4]:
df.describe()

,codigo_ocorrencia,total_recomendacoes
count,6114.000000,6114.000000
mean,59775.058063,0.307164
std,14552.373693,1.143202
min,39115.000000,0.000000
25%,46586.250000,0.000000
50%,53110.500000,0.000000
75%,78142.500000,0.000000
max,80259.000000,23.000000


**Etapa 2: Validação**

In [5]:
pip install pandera

     |████████████████████████████████| 165 kB 4.3 MB/s 


In [6]:
import pandera as pa

In [7]:
# schema de validação de tipos de dados do data frame.

schema = pa.DataFrameSchema(
    columns = {
        "codigo_ocorrencia":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),
        "ocorrencia_cidade":pa.Column(pa.String),
        "ocorrencia_uf":pa.Column(pa.String, pa.Check.str_length(2,2)),
        "ocorrencia_aerodromo":pa.Column(pa.String),
        "ocorrencia_dia":pa.Column(pa.DateTime),
        "ocorrencia_hora":pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9]?$)'), nullable=True),
        "total_recomendacoes":pa.Column(pa.Int)
    }
)

schema.validate(df.head())

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0


**Etapa 3: Limpeza**

In [8]:
# localizando cidade da ocorrencia 02.
df.loc[1, 'ocorrencia_cidade']

'BELÉM'

In [9]:
# localizando matriz das ocorrencias 2,3,4.
df.loc[1:3]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0


In [10]:
# retornando linhas 10 e 40, utilizando lista
df.loc[[10,40]]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
10,39789,INCIDENTE,SÃO PEDRO DO SUL,RS,****,2010-01-10,21:30:00,0
40,39158,INCIDENTE,BELÉM,PA,****,2010-01-28,16:00:00,0


In [11]:
# retornando todas as cidades que tiveram ocorrencias
df.loc[:,'ocorrencia_cidade']

0           RIO DE JANEIRO
1                    BELÉM
2           RIO DE JANEIRO
3       LUCAS DO RIO VERDE
4                  PELOTAS
               ...        
6109                RECIFE
6110           PORTO VELHO
6111                 BELÉM
6112            DOM AQUINO
6113              CAMPINAS
Name: ocorrencia_cidade, Length: 6114, dtype: object

In [12]:
# verificando se todos os valores da coluna sao unicos para definir novo index.
df.codigo_ocorrencia.is_unique

True

In [13]:
# definindo novo index
#df.set_index("codigo_ocorrencia", inplace=True)

# caso deseje retirar o index:
df.reset_index(drop=True, inplace=True)

In [14]:
df.head()

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0


In [15]:
# df.loc[40211]

In [16]:
# renomeando incidentes na cidade de SP para graves
df.loc[df.ocorrencia_uf == 'SP', ['ocorrencia_classificacao']] = 'GRAVE'
df

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...
6109,80245,INCIDENTE,RECIFE,PE,SBRF,2021-08-15,15:05:00,0
6110,80247,INCIDENTE,PORTO VELHO,RO,SBPV,2021-08-16,02:30:00,0
6111,80251,INCIDENTE,BELÉM,PA,SBBE,2021-08-17,15:45:00,0
6112,80257,ACIDENTE,DOM AQUINO,MT,****,2021-08-18,16:00:00,0


In [17]:
# retornando todas as ocorrencias SP
df.loc[df.ocorrencia_uf == 'SP']

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
11,40069,GRAVE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8
16,39809,GRAVE,SÃO PAULO,SP,****,2010-01-15,15:00:00,0
18,39828,GRAVE,SANTOS,SP,****,2010-01-15,17:45:00,0
26,39847,GRAVE,SOROCABA,SP,****,2010-01-20,13:10:00,0
27,39768,GRAVE,CAMPINAS,SP,****,2010-01-21,20:45:00,0
...,...,...,...,...,...,...,...,...
6084,80234,GRAVE,SÃO PAULO,SP,SBSP,2021-07-26,13:20:00,0
6085,80208,GRAVE,SÃO PAULO,SP,SBMT,2021-07-27,19:45:00,0
6104,80238,GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0
6108,80244,GRAVE,BRAGANÇA PAULISTA,SP,SBBP,2021-08-15,15:20:00,0


In [18]:
# procurar todos os dados que nao condizem com os dados das colunas

# ocorrencia_uf
# **

# ocorrencia_arerodromo
# ###!
# ####
# ****
# *****

# ocorrencia_hora
# NULL

In [19]:
df.head()

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0


In [20]:
# substituindo * por <NA> #NA= not available
df.loc[df.ocorrencia_aerodromo == '****', ['ocorrencia_aerodromo']] = pd.NA

In [21]:
#substituindo todos os valores destoantes por <NA>
df.replace(['**','###!','####','****','*****','NULL'], pd.NA, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x
/usr/local/lib/python3.7/dist-packages/pandas/core/missing.py:60: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= arr == x


In [22]:
df

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...
6109,80245,INCIDENTE,RECIFE,PE,SBRF,2021-08-15,15:05:00,0
6110,80247,INCIDENTE,PORTO VELHO,RO,SBPV,2021-08-16,02:30:00,0
6111,80251,INCIDENTE,BELÉM,PA,SBBE,2021-08-17,15:45:00,0
6112,80257,ACIDENTE,DOM AQUINO,MT,<NA>,2021-08-18,16:00:00,0


In [23]:
# contando dados <NA>
df.isna().sum()

codigo_ocorrencia              0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  0
ocorrencia_aerodromo        2272
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [24]:
# contando dados nulos <NULL>
df.isnull().sum()

codigo_ocorrencia              0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  0
ocorrencia_aerodromo        2272
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [25]:
# preenchendo espaços <NA>
df.fillna(000, inplace=True)

In [26]:
# preenchendo valores de apenas uma coluna
# df.fillna(value={'total_recomendacoes':10}, inplace=True)

In [27]:
# criando uma coluna de backup
df['total_recomendacoes_bkp'] = df.total_recomendacoes
df.head()

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,total_recomendacoes_bkp
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,0,2010-01-03,12:00:00,0,0
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,0,2010-01-04,17:30:00,0,0
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0,0


In [28]:
# excluir colunas
df.drop(['total_recomendacoes_bkp'], axis=1, inplace=True)
df.head()

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,0,2010-01-03,12:00:00,0
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,0,2010-01-04,17:30:00,0
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0


In [29]:
# excluir todos os valores nao informados (excluindo linhas)
df.dropna()

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,0,2010-01-03,12:00:00,0
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,0,2010-01-04,17:30:00,0
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...
6109,80245,INCIDENTE,RECIFE,PE,SBRF,2021-08-15,15:05:00,0
6110,80247,INCIDENTE,PORTO VELHO,RO,SBPV,2021-08-16,02:30:00,0
6111,80251,INCIDENTE,BELÉM,PA,SBBE,2021-08-17,15:45:00,0
6112,80257,ACIDENTE,DOM AQUINO,MT,0,2021-08-18,16:00:00,0


In [30]:
# excluir valores faltantes por coluna
df.dropna(subset=['ocorrencia_uf'])

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,0,2010-01-03,12:00:00,0
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,0,2010-01-04,17:30:00,0
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...
6109,80245,INCIDENTE,RECIFE,PE,SBRF,2021-08-15,15:05:00,0
6110,80247,INCIDENTE,PORTO VELHO,RO,SBPV,2021-08-16,02:30:00,0
6111,80251,INCIDENTE,BELÉM,PA,SBBE,2021-08-17,15:45:00,0
6112,80257,ACIDENTE,DOM AQUINO,MT,0,2021-08-18,16:00:00,0


In [31]:
# remover valores duplicados
df.drop_duplicates(inplace=True)

**Etapa4: Transformação**

In [32]:
import numpy as np
import pandas as pd

# dataset retirado no site https://dados.gov.br/dataset/ocorrencias-aeronauticas-da-aviacao-civil-brasileira/resource/9d8a7e09-5f75-47b3-891b-b5f1bab26d59
# Reading dataset as dataframe
valores_ausentes = ['**','###!','####','****','*****','NULL']
df = pd.read_csv("/content/drive/My Drive/data_mining/ocorrencia - ocorrencia.csv", parse_dates=['ocorrencia_dia'], dayfirst=True, na_values=valores_ausentes) # You can also use "utf-8"
pd.set_option('display.max_colwidth', None) # Setting this so we can see the full content of cells
pd.set_option('display.max_columns', None) # to make sure we can see all the columns in output window
df.head(10)

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
5,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0
6,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0


In [33]:
# schema de validação de tipos de dados do data frame.
schema = pa.DataFrameSchema(
    columns = {
        "codigo_ocorrencia":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),
        "ocorrencia_cidade":pa.Column(pa.String),
        "ocorrencia_uf":pa.Column(pa.String, pa.Check.str_length(2,2)),
        "ocorrencia_aerodromo":pa.Column(pa.String, nullable=True), # liberando dados null na coluna
        "ocorrencia_dia":pa.Column(pa.DateTime),
        "ocorrencia_hora":pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9]?$)'), nullable=True),
        "total_recomendacoes":pa.Column(pa.Int)
    }
)

schema.validate(df.head())

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0


In [34]:
# pesquisa pelo label (linha 1), usando loc[]
df.loc[1]

codigo_ocorrencia                         40349
ocorrencia_classificacao              INCIDENTE
ocorrencia_cidade                         BELÉM
ocorrencia_uf                                PA
ocorrencia_aerodromo                       SBBE
ocorrencia_dia              2010-01-03 00:00:00
ocorrencia_hora                        11:05:00
total_recomendacoes                           0
Name: 1, dtype: object

In [35]:
# pesquisa pelo indice, usando iloc[]
df.iloc[-1]

codigo_ocorrencia                         80259
ocorrencia_classificacao              INCIDENTE
ocorrencia_cidade                      CAMPINAS
ocorrencia_uf                                SP
ocorrencia_aerodromo                       SBKP
ocorrencia_dia              2021-08-18 00:00:00
ocorrencia_hora                        12:45:00
total_recomendacoes                           0
Name: 6113, dtype: object

In [36]:
df.tail()

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
6109,80245,INCIDENTE,RECIFE,PE,SBRF,2021-08-15,15:05:00,0
6110,80247,INCIDENTE,PORTO VELHO,RO,SBPV,2021-08-16,02:30:00,0
6111,80251,INCIDENTE,BELÉM,PA,SBBE,2021-08-17,15:45:00,0
6112,80257,ACIDENTE,DOM AQUINO,MT,NaN,2021-08-18,16:00:00,0
6113,80259,INCIDENTE,CAMPINAS,SP,SBKP,2021-08-18,12:45:00,0


In [37]:
# fatiando dados, usando label.
df.loc[10:15]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
10,39789,INCIDENTE,SÃO PEDRO DO SUL,RS,NaN,2010-01-10,21:30:00,0
11,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8
12,40414,INCIDENTE,BRASÍLIA,DF,SBBR,2010-01-10,03:00:00,0
13,39507,ACIDENTE,PRIMAVERA DO LESTE,MT,NaN,2010-01-11,17:00:00,2
14,40107,ACIDENTE,GURUPÁ,PA,SNGU,2010-01-12,14:30:00,0
15,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0


In [38]:
# fatiando dados, usando indice.
df.iloc[10:15]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
10,39789,INCIDENTE,SÃO PEDRO DO SUL,RS,NaN,2010-01-10,21:30:00,0
11,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8
12,40414,INCIDENTE,BRASÍLIA,DF,SBBR,2010-01-10,03:00:00,0
13,39507,ACIDENTE,PRIMAVERA DO LESTE,MT,NaN,2010-01-11,17:00:00,2
14,40107,ACIDENTE,GURUPÁ,PA,SNGU,2010-01-12,14:30:00,0


In [39]:
# busca de dados em uma coluna
df.loc[:,'ocorrencia_uf']

0       RJ
1       PA
2       RJ
3       MT
4       RS
        ..
6109    PE
6110    RO
6111    PA
6112    MT
6113    SP
Name: ocorrencia_uf, Length: 6114, dtype: object

In [40]:
# busca de dados em uma coluna de forma simples
df['ocorrencia_uf']

0       RJ
1       PA
2       RJ
3       MT
4       RS
        ..
6109    PE
6110    RO
6111    PA
6112    MT
6113    SP
Name: ocorrencia_uf, Length: 6114, dtype: object

In [41]:
# retornando is not available
df.isna().sum()

codigo_ocorrencia              0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  0
ocorrencia_aerodromo        2456
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [42]:
# retornando is not null
df.isnull().sum()

codigo_ocorrencia              0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  0
ocorrencia_aerodromo        2456
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [43]:
# filtrar linhas que possuem valores nao disponiveis
df.loc[df.ocorrencia_hora.isnull()]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
4102,78823,INCIDENTE,CORUMBÁ,MS,NaN,2017-07-09,NaN,0


In [44]:
# outra forma de filtrar linhas que possuem valores nao disponiveis
filtro = df.ocorrencia_hora.isnull()
df.loc[filtro]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
4102,78823,INCIDENTE,CORUMBÁ,MS,NaN,2017-07-09,NaN,0


In [45]:
# a funcao count nao conta os valores nulos. É uma forma efetiva de descobrir as colunas que possuem valores faltantes
df.count()

codigo_ocorrencia           6114
ocorrencia_classificacao    6114
ocorrencia_cidade           6114
ocorrencia_uf               6114
ocorrencia_aerodromo        3658
ocorrencia_dia              6114
ocorrencia_hora             6113
total_recomendacoes         6114
dtype: int64

In [46]:
# filtrando ocorrencias com mais de 10 recomensacoes
filtro = df.total_recomendacoes > 10
df.loc[filtro]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
63,39992,ACIDENTE,SÃO PAULO,SP,SBMT,2010-02-17,20:07:00,11
326,42250,ACIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-08-12,12:26:00,19
530,43471,ACIDENTE,SÃO PAULO,SP,NaN,2010-12-14,20:15:00,11
727,44377,ACIDENTE,ITÁPOLIS,SP,SDIO,2011-04-09,14:50:00,11
872,44796,ACIDENTE,RECIFE,PE,SBRF,2011-07-13,09:54:00,23
884,44888,INCIDENTE GRAVE,SÃO JOSÉ DOS PINHAIS,PR,SBCT,2011-07-15,17:05:00,11
1062,45554,INCIDENTE GRAVE,UBERLÂNDIA,MG,SBUL,2011-11-15,02:15:00,20
1972,47938,INCIDENTE,BRASÍLIA,DF,SBBR,2013-04-13,18:00:00,11
2797,52265,ACIDENTE,SANTOS,SP,NaN,2014-08-13,13:03:00,13
3932,66432,INCIDENTE GRAVE,VITÓRIA,ES,NaN,2017-02-21,11:47:00,12


In [47]:
# filtrando cidades que possuem ocorrencias com total de recomendações maior que 10.
filtro = df.total_recomendacoes > 10
df.loc[filtro,['ocorrencia_cidade','total_recomendacoes']]

,ocorrencia_cidade,total_recomendacoes
63,SÃO PAULO,11
326,RIO DE JANEIRO,19
530,SÃO PAULO,11
727,ITÁPOLIS,11
872,RECIFE,23
884,SÃO JOSÉ DOS PINHAIS,11
1062,UBERLÂNDIA,20
1972,BRASÍLIA,11
2797,SANTOS,13
3932,VITÓRIA,12


In [48]:
# filtrando ocorrências com classificação == incidentes graves
filtro = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
df.loc[filtro]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0
29,39709,INCIDENTE GRAVE,CURITIBA,PR,SBBI,2010-01-23,16:36:00,2
35,39487,INCIDENTE GRAVE,PALMAS,TO,SBPJ,2010-01-26,16:50:00,0
...,...,...,...,...,...,...,...,...
6090,80221,INCIDENTE GRAVE,MARAÚ,BA,SIRI,2021-07-31,13:15:00,0
6095,80228,INCIDENTE GRAVE,ELDORADO DO SUL,RS,SIXE,2021-08-05,15:55:00,0
6097,80231,INCIDENTE GRAVE,CAXIAS DO SUL,RS,SBCX,2021-08-06,16:59:00,0
6098,80248,INCIDENTE GRAVE,CHAPADÃO DO SUL,MS,SSCD,2021-08-06,20:00:00,0


In [49]:
# filtrando ocorrências com classificação == INCIDENTE GRAVE no estado de SAO PAULO
filtro1 = df.ocorrencia_classificacao == 'INCIDENTE GRAVE'
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
136,40270,INCIDENTE GRAVE,PIRASSUNUNGA,SP,SDPY,2010-04-15,20:15:00,0
171,40620,INCIDENTE GRAVE,GUARULHOS,SP,SBGR,2010-05-06,15:50:00,0
219,41411,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2010-06-08,19:30:00,2
297,42289,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2010-07-29,20:06:00,0
314,42323,INCIDENTE GRAVE,SÃO JOSÉ DO RIO PRETO,SP,SBSR,2010-08-05,12:01:00,0
...,...,...,...,...,...,...,...,...
5949,80056,INCIDENTE GRAVE,FERNANDÓPOLIS,SP,NaN,2021-04-28,11:20:00,0
6000,80114,INCIDENTE GRAVE,LINS,SP,SWXQ,2021-06-05,19:43:00,0
6007,80124,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2021-06-10,20:30:00,0
6081,80200,INCIDENTE GRAVE,BRAGANÇA PAULISTA,SP,SDVH,2021-07-25,12:25:00,0


In [50]:
# filtrando ocorrências com classificação == INCIDENTE GRAVE ou classificação == INCIDENTE e uf == SP
filtro1 = ((df.ocorrencia_classificacao == 'INCIDENTE GRAVE') | (df.ocorrencia_classificacao == 'INCIDENTE'))
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
16,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0
18,39828,INCIDENTE,SANTOS,SP,NaN,2010-01-15,17:45:00,0
26,39847,INCIDENTE,SOROCABA,SP,NaN,2010-01-20,13:10:00,0
27,39768,INCIDENTE,CAMPINAS,SP,NaN,2010-01-21,20:45:00,0
33,39848,INCIDENTE,GUARULHOS,SP,SBGR,2010-01-25,17:05:00,0
...,...,...,...,...,...,...,...,...
6084,80234,INCIDENTE,SÃO PAULO,SP,SBSP,2021-07-26,13:20:00,0
6085,80208,INCIDENTE,SÃO PAULO,SP,SBMT,2021-07-27,19:45:00,0
6104,80238,INCIDENTE GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0
6108,80244,INCIDENTE,BRAGANÇA PAULISTA,SP,SBBP,2021-08-15,15:20:00,0


In [51]:
# Outra forma de filtrar ocorrências com classificação == INCIDENTE GRAVE ou classificação == INCIDENTE e uf == SP
filtro1 = df.ocorrencia_classificacao.isin(['INCIDENTE GRAVE', 'INCIDENTE'])
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
16,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0
18,39828,INCIDENTE,SANTOS,SP,NaN,2010-01-15,17:45:00,0
26,39847,INCIDENTE,SOROCABA,SP,NaN,2010-01-20,13:10:00,0
27,39768,INCIDENTE,CAMPINAS,SP,NaN,2010-01-21,20:45:00,0
33,39848,INCIDENTE,GUARULHOS,SP,SBGR,2010-01-25,17:05:00,0
...,...,...,...,...,...,...,...,...
6084,80234,INCIDENTE,SÃO PAULO,SP,SBSP,2021-07-26,13:20:00,0
6085,80208,INCIDENTE,SÃO PAULO,SP,SBMT,2021-07-27,19:45:00,0
6104,80238,INCIDENTE GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0
6108,80244,INCIDENTE,BRAGANÇA PAULISTA,SP,SBBP,2021-08-15,15:20:00,0


In [52]:
# filtrando todas as ocorrencias nas quais as cidades comecam com letra 'C'
filtro = df.ocorrencia_cidade.str[0] =='C'
df.loc[filtro]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
6,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
15,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
20,40310,INCIDENTE,CRUZEIRO DO SUL,AC,SBCZ,2010-01-18,19:32:00,0
...,...,...,...,...,...,...,...,...
6094,80225,INCIDENTE,CURITIBA,PR,SBBI,2021-08-04,13:45:00,0
6097,80231,INCIDENTE GRAVE,CAXIAS DO SUL,RS,SBCX,2021-08-06,16:59:00,0
6098,80248,INCIDENTE GRAVE,CHAPADÃO DO SUL,MS,SSCD,2021-08-06,20:00:00,0
6100,80235,INCIDENTE,CORUMBÁ,MS,SBCR,2021-08-09,20:55:00,0


In [53]:
# filtrando todas as ocorrencias nas quais as cidades terminam com letra 'A'
filtro = df.ocorrencia_cidade.str[-1] == 'A'
df.loc[filtro]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
12,40414,INCIDENTE,BRASÍLIA,DF,SBBR,2010-01-10,03:00:00,0
15,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
25,39772,INCIDENTE,UBERLÂNDIA,MG,NaN,2010-01-20,18:58:00,0
26,39847,INCIDENTE,SOROCABA,SP,NaN,2010-01-20,13:10:00,0
...,...,...,...,...,...,...,...,...
6093,80227,ACIDENTE,UMUARAMA,PR,NaN,2021-08-02,14:30:00,0
6094,80225,INCIDENTE,CURITIBA,PR,SBBI,2021-08-04,13:45:00,0
6102,80240,INCIDENTE,GOIÂNIA,GO,SBNV,2021-08-10,14:15:00,0
6104,80238,INCIDENTE GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0


In [54]:
# filtrando todas as ocorrencias nas quais as cidades terminam com os caracteres 'NA'
filtro = df.ocorrencia_cidade.str[-2:] == 'NA'
df.loc[filtro]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
51,39115,ACIDENTE,CORRENTINA,BA,SNTY,2010-02-07,17:40:00,2
81,39914,INCIDENTE,LONDRINA,PR,SBLO,2010-03-09,21:00:00,0
86,39993,INCIDENTE GRAVE,LONDRINA,PR,SBLO,2010-03-11,12:30:00,2
138,40328,ACIDENTE,CORRENTINA,BA,NaN,2010-04-15,17:25:00,0
147,40321,INCIDENTE,LONDRINA,PR,NaN,2010-04-20,16:11:00,0
...,...,...,...,...,...,...,...,...
5871,79961,ACIDENTE,AMERICANA,SP,SDAI,2021-03-09,19:30:00,0
5875,79976,ACIDENTE,ARAGUAIANA,MT,NaN,2021-03-10,20:30:00,0
5899,79991,INCIDENTE,AMERICANA,SP,SDAI,2021-03-25,17:35:00,0
6012,80134,INCIDENTE,TERESINA,PI,SBTE,2021-06-16,04:42:00,0


In [55]:
# filtrando todas as ocorrencias que contém os caracteres 'MA' nas cidades
filtro = df.ocorrencia_cidade.str.contains('MA')
df.loc[filtro]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
13,39507,ACIDENTE,PRIMAVERA DO LESTE,MT,NaN,2010-01-11,17:00:00,2
15,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
35,39487,INCIDENTE GRAVE,PALMAS,TO,SBPJ,2010-01-26,16:50:00,0
78,39916,INCIDENTE GRAVE,MANAUS,AM,SBEG,2010-03-07,19:35:00,2
...,...,...,...,...,...,...,...,...
6075,80205,INCIDENTE,MANAUS,AM,SBEG,2021-07-23,11:15:00,0
6086,80212,INCIDENTE,AMAPÁ,AP,NaN,2021-07-28,18:00:00,0
6090,80221,INCIDENTE GRAVE,MARAÚ,BA,SIRI,2021-07-31,13:15:00,0
6093,80227,ACIDENTE,UMUARAMA,PR,NaN,2021-08-02,14:30:00,0


In [56]:
# filtrando todas as ocorrencias que contém os caracteres 'MA' ou 'AL' nas cidades
filtro = df.ocorrencia_cidade.str.contains('MA|AL')
df.loc[filtro]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
5,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0
7,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
13,39507,ACIDENTE,PRIMAVERA DO LESTE,MT,NaN,2010-01-11,17:00:00,2
15,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
17,39810,INCIDENTE,SALVADOR,BA,NaN,2010-01-15,18:23:00,0
...,...,...,...,...,...,...,...,...
6075,80205,INCIDENTE,MANAUS,AM,SBEG,2021-07-23,11:15:00,0
6086,80212,INCIDENTE,AMAPÁ,AP,NaN,2021-07-28,18:00:00,0
6090,80221,INCIDENTE GRAVE,MARAÚ,BA,SIRI,2021-07-31,13:15:00,0
6093,80227,ACIDENTE,UMUARAMA,PR,NaN,2021-08-02,14:30:00,0


In [57]:
# filtrando ocorrencias do ano == 2020
filtro = df.ocorrencia_dia.dt.year == 2020
df.loc[filtro]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
5243,79043,ACIDENTE,CAMANDUCAIA,MG,SNEJ,2020-01-01,21:00:00,1
5244,79044,INCIDENTE GRAVE,SALVADOR,BA,SBSV,2020-01-03,16:35:00,0
5245,79049,ACIDENTE,LEME,SP,NaN,2020-01-03,21:15:00,0
5246,79132,INCIDENTE,SÃO MATEUS,ES,NaN,2020-01-03,14:10:00,0
5247,79182,INCIDENTE,GOIÂNIA,GO,SBGO,2020-01-03,11:10:00,0
...,...,...,...,...,...,...,...,...
5748,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5749,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,NaN,2020-12-30,18:30:00,0
5750,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5751,79756,INCIDENTE GRAVE,VICENTINA,MS,NaN,2020-12-31,09:00:00,0


In [58]:
# filtrando ocorrencias de dezembro/2020
filtro1 = df.ocorrencia_dia.dt.year == 2020
filtro2 = df.ocorrencia_dia.dt.month == 12
df.loc[filtro1 & filtro2]

# ou
# filtro = (df.ocorrencia_dia.dt.year == 2020) & (df.ocorrencia_dia.dt.month == 12)
# df.loc[filtro]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
5702,79750,INCIDENTE,GOIÂNIA,GO,SBNV,2020-12-01,11:30:00,0
5703,79701,ACIDENTE,QUATRO BARRAS,PR,NaN,2020-12-02,16:35:00,0
5704,79702,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-02,16:30:00,0
5705,79739,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2020-12-03,14:42:00,0
5706,79705,INCIDENTE GRAVE,SOROCABA,SP,SDCO,2020-12-04,15:30:00,0
5707,79707,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2020-12-05,12:25:00,0
5708,79712,INCIDENTE GRAVE,BOM JESUS,PI,NaN,2020-12-06,11:00:00,0
5709,79713,ACIDENTE,SANTA VITÓRIA DO PALMAR,RS,NaN,2020-12-06,13:30:00,3
5710,79711,INCIDENTE,PORTO VELHO,RO,SBPV,2020-12-08,04:23:00,0
5711,79714,INCIDENTE,RIO DE JANEIRO,RJ,9PAA,2020-12-08,14:50:00,0


In [59]:
# filtrando ocorrencias de 03/dezembro/2020
filtro1 = df.ocorrencia_dia.dt.year == 2020
filtro2 = df.ocorrencia_dia.dt.month == 12
filtro3 = df.ocorrencia_dia.dt.day == 8
df.loc[filtro1 & filtro2 & filtro3]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
5710,79711,INCIDENTE,PORTO VELHO,RO,SBPV,2020-12-08,04:23:00,0
5711,79714,INCIDENTE,RIO DE JANEIRO,RJ,9PAA,2020-12-08,14:50:00,0
5712,79717,ACIDENTE,NÃO IDENTIFICADA,MT,NaN,2020-12-08,17:00:00,0
5713,79725,INCIDENTE,LONDRINA,PR,SBLO,2020-12-08,18:55:00,0


In [60]:
# filtrando ocorrencias do dias 03/dezembro/2020 ate 08/dezembro/2020
filtro1 = df.ocorrencia_dia.dt.year == 2020
filtro2 = df.ocorrencia_dia.dt.month == 12
filtro3 = (df.ocorrencia_dia.dt.day >=3) & (df.ocorrencia_dia.dt.day <=8)
df.loc[filtro1 & filtro2 & filtro3]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
5705,79739,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2020-12-03,14:42:00,0
5706,79705,INCIDENTE GRAVE,SOROCABA,SP,SDCO,2020-12-04,15:30:00,0
5707,79707,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2020-12-05,12:25:00,0
5708,79712,INCIDENTE GRAVE,BOM JESUS,PI,NaN,2020-12-06,11:00:00,0
5709,79713,ACIDENTE,SANTA VITÓRIA DO PALMAR,RS,NaN,2020-12-06,13:30:00,3
5710,79711,INCIDENTE,PORTO VELHO,RO,SBPV,2020-12-08,04:23:00,0
5711,79714,INCIDENTE,RIO DE JANEIRO,RJ,9PAA,2020-12-08,14:50:00,0
5712,79717,ACIDENTE,NÃO IDENTIFICADA,MT,NaN,2020-12-08,17:00:00,0
5713,79725,INCIDENTE,LONDRINA,PR,SBLO,2020-12-08,18:55:00,0


In [61]:
# criando nova coluna com data e hora juntos
df['ocorrencia_dia_hora'] = pd.to_datetime(df.ocorrencia_dia.astype(str) + ' ' + df.ocorrencia_hora)
df.head()

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0,2010-01-03 12:00:00
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0,2010-01-03 11:05:00
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,2010-01-03 03:00:00
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0,2010-01-04 17:30:00
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0,2010-01-05 19:25:00


In [62]:
# bucando data e hora de uma ocorrencia
filtro1 = df.ocorrencia_dia_hora >= '2010-01-03 11:00:00'
filtro2 = df.ocorrencia_dia_hora <= '2010-01-05 20:00:00'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0,2010-01-03 12:00:00
1,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0,2010-01-03 11:05:00
3,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0,2010-01-04 17:30:00
4,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0,2010-01-05 19:25:00


In [63]:
# criando um novo dataframe em forma de agrupamento de dados. Ocorrencia de 2020, mes 03
filtro1 = df.ocorrencia_dia.dt.year == 2020
filtro2 = df.ocorrencia_dia.dt.month == 3
df_ano202003 = df.loc[filtro1 & filtro2]
df_ano202003

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
5348,79207,INCIDENTE GRAVE,CUIABÁ,MT,SBCY,2020-03-01,23:00:00,0,2020-03-01 23:00:00
5349,79212,ACIDENTE,VIANA,ES,NaN,2020-03-01,15:12:00,0,2020-03-01 15:12:00
5350,79217,INCIDENTE,SÃO LUÍS,MA,NaN,2020-03-01,19:40:00,0,2020-03-01 19:40:00
5351,79214,INCIDENTE,CAMPOS DOS GOYTACAZES,RJ,NaN,2020-03-02,12:50:00,0,2020-03-02 12:50:00
5352,79216,INCIDENTE,BELO HORIZONTE,MG,SBBH,2020-03-03,18:25:00,0,2020-03-03 18:25:00
5353,79462,INCIDENTE,RIBEIRÃO PRETO,SP,SBRP,2020-03-03,19:42:00,0,2020-03-03 19:42:00
5354,79215,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2020-03-04,21:30:00,0,2020-03-04 21:30:00
5355,79225,INCIDENTE,MANTENA,MG,NaN,2020-03-04,12:04:00,0,2020-03-04 12:04:00
5356,79292,INCIDENTE,CAMPINAS,SP,NaN,2020-03-04,22:50:00,0,2020-03-04 22:50:00
5357,79213,ACIDENTE,RIO CLARO,SP,SDRK,2020-03-05,20:00:00,5,2020-03-05 20:00:00


In [64]:
df_ano202003.count()

codigo_ocorrencia           44
ocorrencia_classificacao    44
ocorrencia_cidade           44
ocorrencia_uf               44
ocorrencia_aerodromo        17
ocorrencia_dia              44
ocorrencia_hora             44
total_recomendacoes         44
ocorrencia_dia_hora         44
dtype: int64

In [65]:
df_ano202003.groupby(['codigo_ocorrencia']).codigo_ocorrencia.count()

codigo_ocorrencia
79207    1
79212    1
79213    1
79214    1
79215    1
79216    1
79217    1
79220    1
79225    1
79228    1
79230    1
79231    1
79232    1
79233    1
79234    1
79235    1
79236    1
79237    1
79238    1
79242    1
79243    1
79244    1
79245    1
79246    1
79247    1
79248    1
79249    1
79250    1
79251    1
79252    1
79253    1
79256    1
79257    1
79265    1
79284    1
79286    1
79287    1
79288    1
79289    1
79292    1
79297    1
79314    1
79462    1
79710    1
Name: codigo_ocorrencia, dtype: int64

In [66]:
# contando por agrupamento de dados
df_ano202003.groupby(['ocorrencia_classificacao']).codigo_ocorrencia.count()

ocorrencia_classificacao
ACIDENTE           12
INCIDENTE          27
INCIDENTE GRAVE     5
Name: codigo_ocorrencia, dtype: int64

In [67]:
df_ano202003.groupby(['ocorrencia_classificacao']).size()

ocorrencia_classificacao
ACIDENTE           12
INCIDENTE          27
INCIDENTE GRAVE     5
dtype: int64

In [68]:
# ordenando valores
df_ano202003.groupby(['ocorrencia_classificacao']).size().sort_values(ascending=True)
# df_ano202003.groupby(['ocorrencia_classificacao']).size().sort_values(ascending=False)

ocorrencia_classificacao
INCIDENTE GRAVE     5
ACIDENTE           12
INCIDENTE          27
dtype: int64

In [71]:
# agrupamento na regiao sudeste ano 2010
filtro1 = df.ocorrencia_dia.dt.year == 2010
filtro2 = df.ocorrencia_uf.isin(['SP','MG','ES','RJ'])
df_sudeste2010 = df.loc[filtro1 & filtro2]
df_sudeste2010

,codigo_ocorrencia,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
0,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0,2010-01-03 12:00:00
2,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,2010-01-03 03:00:00
9,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0,2010-01-10 20:00:00
11,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8,2010-01-10 14:50:00
16,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0,2010-01-15 15:00:00
...,...,...,...,...,...,...,...,...,...
541,43722,ACIDENTE,CAJOBI,SP,NaN,2010-12-23,11:00:00,0,2010-12-23 11:00:00
544,43548,INCIDENTE,SÃO PAULO,SP,SBSP,2010-12-26,22:10:00,0,2010-12-26 22:10:00
548,43724,ACIDENTE,COSMÓPOLIS,SP,NaN,2010-12-29,12:00:00,0,2010-12-29 12:00:00
550,43730,INCIDENTE,VITÓRIA,ES,NaN,2010-12-30,18:10:00,0,2010-12-30 18:10:00


In [72]:
# agrupamento incidente na regiao sudeste
df_sudeste2010.groupby(['ocorrencia_classificacao']).size()

ocorrencia_classificacao
ACIDENTE            40
INCIDENTE          170
INCIDENTE GRAVE     18
dtype: int64

In [73]:
# para validar a linha de codigo 72, contamos o total das ocorrencias. Veremos total de 228 ocorr.
df_sudeste2010.count()

codigo_ocorrencia           228
ocorrencia_classificacao    228
ocorrencia_cidade           228
ocorrencia_uf               228
ocorrencia_aerodromo        127
ocorrencia_dia              228
ocorrencia_hora             228
total_recomendacoes         228
ocorrencia_dia_hora         228
dtype: int64

In [76]:
# agrupamento incidente na regiao sudeste por estado
df_sudeste2010.groupby(['ocorrencia_classificacao','ocorrencia_uf']).size()
#df_sudeste2010.groupby(['ocorrencia_uf','ocorrencia_classificacao']).size()

ocorrencia_classificacao  ocorrencia_uf
ACIDENTE                  MG                7
                          RJ                9
                          SP               24
INCIDENTE                 ES                5
                          MG               39
                          RJ               52
                          SP               74
INCIDENTE GRAVE           MG                5
                          RJ                4
                          SP                9
dtype: int64

In [77]:
# agrupamento incidente na regiao sudeste por cidade em forma crescente
df_sudeste2010.groupby(['ocorrencia_cidade']).size().sort_values(ascending=False)

ocorrencia_cidade
RIO DE JANEIRO           47
SÃO PAULO                33
GUARULHOS                18
BELO HORIZONTE           17
CAMPINAS                 13
MACAÉ                    12
CONFINS                   7
UBERLÂNDIA                5
VITÓRIA                   4
JUIZ DE FORA              4
SÃO JOSÉ DO RIO PRETO     3
SÃO JOSÉ DOS CAMPOS       3
MARÍLIA                   3
ARARAQUARA                2
MONTES CLAROS             2
CAMPOS DOS GOYTACAZES     2
DIVINÓPOLIS               2
IPATINGA                  2
ITANHAÉM                  2
JUNDIAÍ                   2
POÇOS DE CALDAS           2
RIBEIRÃO PRETO            2
SOROCABA                  2
UBERABA                   2
BAURU                     2
AREALVA                   2
BOITUVA                   1
BOCAINA                   1
BOM JESUS DO AMPARO       1
BOM JESUS DO GALHO        1
BEBEDOURO                 1
BRAGANÇA PAULISTA         1
BROTAS                    1
CAJOBI                    1
ARRAIAL DO CABO           1
CA

In [79]:
# contando total de recomendacoes que o estado do RIO DE JANEIRO registrou.
filtro = df_sudeste2010.ocorrencia_cidade == 'RIO DE JANEIRO'
df_sudeste2010.loc[filtro].total_recomendacoes.sum()

25

In [80]:
# total de recomendacoes por cidade da regiao sudeste
df_sudeste2010.groupby(['ocorrencia_cidade']).total_recomendacoes.sum()

ocorrencia_cidade
AMERICANA                 0
ARARAQUARA                0
ARAÇATUBA                 0
AREALVA                   0
ARRAIAL DO CABO           0
BAURU                     0
BEBEDOURO                 0
BELO HORIZONTE            2
BOCAINA                   0
BOITUVA                   1
BOM JESUS DO AMPARO       0
BOM JESUS DO GALHO        1
BRAGANÇA PAULISTA         9
BROTAS                    0
CAJOBI                    0
CAMANDUCAIA               9
CAMPINAS                  0
CAMPOS DOS GOYTACAZES     0
CASIMIRO DE ABREU         0
CONFINS                   0
COROMANDEL                0
COSMÓPOLIS                0
DIAMANTINA                0
DIVINÓPOLIS               0
GUARULHOS                 0
IPATINGA                  0
IPERÓ                     3
ITANHAÉM                  1
ITÁPOLIS                  0
JUIZ DE FORA              0
JUNDIAÍ                   7
LINHARES                  0
MACAÉ                     0
MANGARATIBA               0
MARÍLIA                   0
MO

In [83]:
# agrupando apenas as cidade com recomendacoes
filtro = df_sudeste2010.total_recomendacoes > 0
df_sudeste2010.loc[filtro].groupby(['ocorrencia_cidade']).total_recomendacoes.sum().sort_values()

ocorrencia_cidade
BOITUVA                   1
BOM JESUS DO GALHO        1
ITANHAÉM                  1
BELO HORIZONTE            2
SÃO BERNARDO DO CAMPO     2
IPERÓ                     3
NOVO HORIZONTE            3
TIMÓTEO                   3
JUNDIAÍ                   7
RESENDE                   8
BRAGANÇA PAULISTA         9
CAMANDUCAIA               9
RIO DE JANEIRO           25
SÃO PAULO                41
Name: total_recomendacoes, dtype: int64

In [84]:
# agrupar ocorrencias por cidade e por mês
filtro = df_sudeste2010.total_recomendacoes > 0
df_sudeste2010.loc[filtro].groupby(['ocorrencia_cidade', df_sudeste2010.ocorrencia_dia.dt.month]).total_recomendacoes.sum()

ocorrencia_cidade      ocorrencia_dia
BELO HORIZONTE         2                  1
                       3                  1
BOITUVA                7                  1
BOM JESUS DO GALHO     12                 1
BRAGANÇA PAULISTA      11                 9
CAMANDUCAIA            7                  9
IPERÓ                  1                  3
ITANHAÉM               9                  1
JUNDIAÍ                5                  5
                       6                  2
NOVO HORIZONTE         7                  3
RESENDE                4                  8
RIO DE JANEIRO         4                  3
                       7                  3
                       8                 19
SÃO BERNARDO DO CAMPO  1                  2
SÃO PAULO              1                  8
                       2                 14
                       6                  2
                       11                 6
                       12                11
TIMÓTEO                2              

**Obrigada!**